The first step is to getting an AI-generated data visualization is to get a description of the dataset. This description will be used as a guide for Retrieval Augmented Generation over the <a href="https://airbnb.io/visx/">visx</a> library documentation. For simplicity, we will limit the data format to CSV files. In this notebook we will use OpenAI's assistants and the code interpreter tool to get a comprehensive description of a CSV file.

We start with importing the `openai` and `python-dotenv` libraries. An `OPENAI_API_KEY` should be defined in a `.env` file.

In [11]:
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
load_dotenv()

True

Now we need to load a sample dataset. I will be using a large sales dataset from kaggle (source: https://www.kaggle.com/datasets/seiffathi/sales-analysis-datasets)

This is what the dataset looks like:

In [77]:
df=pd.read_csv("../data/sample.csv")
print("Number of rows:",df.shape[0], "Number of columns:", df.shape[1], "Column names:",list(df.columns))

Number of rows: 18383 Number of columns: 6 Column names: ['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address']


In [76]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


Now we need to define a prompt for the assistant to use. This prompt needs to guide the assistant to use the pandas library to inspect the uploaded CSV file and get the request description.

In [92]:
instructions="""
You're a skilled Python programmer tasked with creating Python 3 solutions for analyzing pandas dataframes, following top industry practices. 
Help the user analyze their data, using code to assist yourself. 

Make sure your code complies with these rules:
1. Plan first: Have a clear strategy before you start. Outline your approach if it helps.
2. Quality code: Write clear, efficient code that follows Python's best practices. Aim for clean, easy-to-read, and maintainable code.
3. Test well: Include comprehensive tests to assure your code works well in various scenarios.

Additionally, your response must follow these guidelines:
1. If the user asks for a visualization, suggest a method but do not write any code to visualiza the data for the user. Examples of your response may be:
Bar chart
Histogram
Pie chart
Your response may be another visualization type as well. 
2. Keep your responses to the user concise. No yapping. 
3. Only state information that directly answers the users query.
4. Focus on what the data represents. If the user asks for a datatype of a column, infer what it should be based of column name and date. 
Example: 
column name: Order date
column value: 04/07/19 22:30
datatype: date
"""

Now initializing an openAI client with the prompt and sample csv file:

In [93]:
client=OpenAI() 
file=client.files.create(
  file=open("../data/sample.csv", "rb"),
  purpose='assistants'
)
file_id=file.id

In [94]:
assistant = client.beta.assistants.create(
  name="data analyst 2",
  instructions=instructions,
  tools=[{"type": "code_interpreter"}],
temperature=0.2,
  tool_resources={
    "code_interpreter": {
      "file_ids": [file_id]
    }},
  model="gpt-4o",
)
assistant_id=assistant.id

Now we create an empty thread. This thread will store the user prompt.

In [95]:
thread = client.beta.threads.create()
thread_id=thread.id

In [96]:
message = client.beta.threads.messages.create(
  thread_id=thread_id,
  role="user",
  content="Provide a summary of the dataset. List its length, the categories and their datatypes. State what the dataset represents. Suggest one data visualization method suitable for this dataset."
)

In [97]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread_id,
  assistant_id=assistant_id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Let's start by loading the dataset and examining its structure to provide a summary. We'll look at the first few rows, the column names, their datatypes, and the length of the dataset. This will help us understand what the dataset represents and suggest a suitable visualization method.
assistant > code_interpreter

import pandas as pd

# Load the dataset
file_path = '/mnt/data/file-Ixo74dUkLx6unkqip3NIFwBg'
df = pd.read_csv(file_path)

# Display the first few rows, column names, datatypes, and length of the dataset
df_head = df.head()
df_info = df.info()
df_length = len(df)

df_head, df_info, df_length

output >

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18383 entries, 0 to 18382
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Order ID          18324 non-null  object
 1   Product           18324 non-null  object
 2   Quantity Ordered  18324 non-null  object
 3   Price Each        18324

The relevant information is in the last message of the thread, so we will retrieve and store this

In [103]:
thread_messages = client.beta.threads.messages.list(thread_id)
data=thread_messages.data

In [104]:
len(data)

3

In [111]:
last_message=data[0]
last_message_content=last_message.content[0].text.value

In [113]:
print(last_message_content)

### Summary of the Dataset

- **Length**: 18,383 entries
- **Columns and Datatypes**:
  - `Order ID`: object (should be an identifier)
  - `Product`: object (product name)
  - `Quantity Ordered`: object (should be integer)
  - `Price Each`: object (should be float)
  - `Order Date`: object (should be date)
  - `Purchase Address`: object (address)

### Dataset Representation
The dataset appears to represent sales data, including information about orders, products, quantities, prices, order dates, and purchase addresses.

### Suggested Data Visualization
A suitable visualization method for this dataset could be a **bar chart** to show the total sales per product.


Since these runs are expensive, I was curious to see if we can use the OpenAI completions API to generate the necessary Python code and then  run this code on the local file. First we will need a prompt and a custom function. The aim of the custom function is to generate code which we will then execute.

In [45]:
df_analysis_function=[
    {
        "type": "function",
        "function": {
            "name": "write_pandas_code",
            "description": "Write Python 3 Pandas script that loads the dataset from the path provided in the import text and defines a function which when called performs analysis required by the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "func_name":{
                        "type": "string", 
                        "description":"The name of the function you have defined"
                    },
                    "script": {
                        "type": "string",
                        "description": "The script to perform the analysis requested by the user"
                    }
                }
            }
        }
    }
]

system_prompt="""
You're a skilled Python programmer tasked with creating Python 3 solutions for analyzing data, following top industry practices. Your task is to write a python script that loads the file from the file path in the input text and defines a function which performs the analysis requested by the user using the pandas library. The function must be defined but must not be executed in the script. The function must get all requested information in one call. The function must both print and return the results.
"""
file_path="../data/sample.csv"
user_prompt=f"""My file is located at {file_path}. Please write a script that gets the length and size of the dataset, the column names, and their datatypes and a dictionary with a key as the column name and a random sample as the value. The script must return all requested information compiled as a dictionary."""


In [46]:
client=OpenAI() #reinitialize the client here
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
    temperature=0.2, 
    tools=df_analysis_function, 
    tool_choice="required"

)
response=completion.choices[0].message.tool_calls[0].function.arguments
response

'{"func_name": "analyze_dataset", "script": "import pandas as pd\\nimport numpy as np\\n\\ndef analyze_dataset(file_path):\\n    # Load the dataset\\n    df = pd.read_csv(file_path)\\n    \\n    # Get the length and size of the dataset\\n    length = len(df)\\n    size = df.size\\n    \\n    # Get the column names and their datatypes\\n    columns = df.columns.tolist()\\n    dtypes = df.dtypes.to_dict()\\n    \\n    # Get a random sample for each column\\n    random_samples = {col: df[col].sample(1).iloc[0] for col in df.columns}\\n    \\n    # Compile the results into a dictionary\\n    results = {\\n        \'length\': length,\\n        \'size\': size,\\n        \'columns\': columns,\\n        \'dtypes\': dtypes,\\n        \'random_samples\': random_samples\\n    }\\n    \\n    # Print the results\\n    print(results)\\n    \\n    return results"}'

In [47]:
import json
parsed_response=json.loads(response)
func_name=parsed_response["func_name"]
ai_code=parsed_response['script']
ai_code

"import pandas as pd\nimport numpy as np\n\ndef analyze_dataset(file_path):\n    # Load the dataset\n    df = pd.read_csv(file_path)\n    \n    # Get the length and size of the dataset\n    length = len(df)\n    size = df.size\n    \n    # Get the column names and their datatypes\n    columns = df.columns.tolist()\n    dtypes = df.dtypes.to_dict()\n    \n    # Get a random sample for each column\n    random_samples = {col: df[col].sample(1).iloc[0] for col in df.columns}\n    \n    # Compile the results into a dictionary\n    results = {\n        'length': length,\n        'size': size,\n        'columns': columns,\n        'dtypes': dtypes,\n        'random_samples': random_samples\n    }\n    \n    # Print the results\n    print(results)\n    \n    return results"

In [48]:
func_name

'analyze_dataset'

Now we can run the code to define the function, and perform the analysis on our dataset. Then by doing so we get the output as a dictionary and are then able to feed it to the model in another API call.

In [49]:
exec(ai_code)

In [52]:
data_analysis=analyze_dataset(file_path)


{'length': 18383, 'size': 110298, 'columns': ['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address'], 'dtypes': {'Order ID': dtype('O'), 'Product': dtype('O'), 'Quantity Ordered': dtype('O'), 'Price Each': dtype('O'), 'Order Date': dtype('O'), 'Purchase Address': dtype('O')}, 'random_samples': {'Order ID': '186198', 'Product': 'AAA Batteries (4-pack)', 'Quantity Ordered': '1', 'Price Each': '11.95', 'Order Date': '04/30/19 21:13', 'Purchase Address': '764 Adams St, San Francisco, CA 94016'}}


The output can be more structured, however this works for testing as we will be passing the return values as a string in another API call. Now we need a new prompt that introduces the `data_analysis` summary to the model and asks it to perform a qualitative analysis inferring any missing information. We also need to define a function to get the output from this call in a structured format

In [62]:
format_tools=[
    {
        "type": "function",
        "function": {
            "name": "summarize_dataset",
            "description": "Summarize the dataset provided in the input text",
            "parameters": {
                "type": "object",
                "properties": {
                "summary": {
                    "type": "string",
                    "description": "A statement of what the dataset seems to represent. Use the column names to guide you"
                },
                "suggested_visualization": {
                    "type": "string",
                    "description": "One way the dataset may be visualized. Examples include: bar chart, pie chart"
                },
                "datatypes": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "column_name": {
                                    "type": "string",
                                    "description": "The name of the column"
                                },
                                "datatype": {
                                    "type": "string",
                                    "description": "The datatype best suited to the column. Use the examples in the input text to guide you"
                                }
                            }
                        }
                    }
                }
            }
        }
    }
]


In [63]:
system_prompt="""
You are an experienced data analyst provided with a summary of the dataset. From the summary extract the information requested by the user. Where you are unsure, take a guess using column names and provided examples to guide you. Keep your response concise.
"""
user_prompt=f"""here is my dataset: {str(data_analysis)}"""
user_prompt

"here is my dataset: {'length': 18383, 'size': 110298, 'columns': ['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date', 'Purchase Address'], 'dtypes': {'Order ID': dtype('O'), 'Product': dtype('O'), 'Quantity Ordered': dtype('O'), 'Price Each': dtype('O'), 'Order Date': dtype('O'), 'Purchase Address': dtype('O')}, 'random_samples': {'Order ID': '186198', 'Product': 'AAA Batteries (4-pack)', 'Quantity Ordered': '1', 'Price Each': '11.95', 'Order Date': '04/30/19 21:13', 'Purchase Address': '764 Adams St, San Francisco, CA 94016'}}"

In [65]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
    temperature=0.2, 
    tools=format_tools, 
    tool_choice="required"

)
response=completion.choices[0].message.tool_calls[0].function.arguments
response

'{"summary": "The dataset appears to represent sales order data, including details about the order ID, product, quantity ordered, price per item, order date, and purchase address.", "suggested_visualization": "bar chart", "datatypes": [{"column_name": "Order ID", "datatype": "string"}, {"column_name": "Product", "datatype": "string"}, {"column_name": "Quantity Ordered", "datatype": "string"}, {"column_name": "Price Each", "datatype": "string"}, {"column_name": "Order Date", "datatype": "string"}, {"column_name": "Purchase Address", "datatype": "string"}]}'

In [67]:
json_response=json.loads(response)
json_response

{'summary': 'The dataset appears to represent sales order data, including details about the order ID, product, quantity ordered, price per item, order date, and purchase address.',
 'suggested_visualization': 'bar chart',
 'datatypes': [{'column_name': 'Order ID', 'datatype': 'string'},
  {'column_name': 'Product', 'datatype': 'string'},
  {'column_name': 'Quantity Ordered', 'datatype': 'string'},
  {'column_name': 'Price Each', 'datatype': 'string'},
  {'column_name': 'Order Date', 'datatype': 'string'},
  {'column_name': 'Purchase Address', 'datatype': 'string'}]}